<a href="https://colab.research.google.com/github/zohaibkhanzohaibi/NIS-CCP/blob/main/Known_Plaintext_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Known-Plaintext Attack for the composite cipher
from collections import defaultdict
import math

ALPH = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
A2I = {c: i for i, c in enumerate(ALPH)}
I2A = {i: c for i, c in enumerate(ALPH)}

def normalize(s):
    return "".join(ch for ch in s.upper() if ch.isalpha())

# composite decrypt (same as earlier): reverse order
def caesar_shift_text(text, shift):
    return "".join(I2A[(A2I[ch] + shift) % 26] for ch in text)

def vigenere_decrypt(text, key):
    out = []
    for i, ch in enumerate(text):
        k = key[i % len(key)]
        out.append(I2A[(A2I[ch] - k) % 26])
    return "".join(out)

def composite_decrypt(ciphertext, key_letters):
    text = normalize(ciphertext)
    key = key_letters
    for kr in reversed(key):
        text = vigenere_decrypt(text, key)
        text = caesar_shift_text(text, -kr)
    return text

# ---------- modular helpers ----------
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def inv_mod(a, m):
    g, x, _ = egcd(a % m, m)
    if g != 1:
        return None
    return x % m

def solve_congruence_for_k_t(Keff_minus_St, m):
    """
    Solve m * k_t ≡ Keff_minus_St (mod 26) for k_t.
    Returns list of solutions in 0..25.
    """
    rhs = Keff_minus_St % 26
    g = math.gcd(m, 26)
    if rhs % g != 0:
        return []  # no solution
    m_r = m // g
    mod_r = 26 // g
    rhs_r = (rhs // g) % mod_r
    inv = inv_mod(m_r, mod_r)
    sols = []
    if inv is None:
        # fallback enumeration
        for cand in range(26):
            if (m * cand - Keff_minus_St) % 26 == 0:
                sols.append(cand % 26)
    else:
        k0 = (rhs_r * inv) % mod_r
        # lift to g solutions
        for t in range(g):
            sols.append((k0 + t * mod_r) % 26)
    return sorted(set(sols))

# ---------- Known-plaintext solver ----------
def recover_keys_from_known_plaintext(known_plaintext, known_ciphertext, m):
    """
    known_plaintext, known_ciphertext: strings (should align; non-letters ignored)
    m: assumed key length (period)
    Returns list of candidate keys (each key is list of ints 0..25).
    """
    P = normalize(known_plaintext)
    C = normalize(known_ciphertext)
    if len(P) != len(C):
        raise ValueError("Known plaintext and ciphertext must align and have same letter count.")

    # Build per-residue observed effective shift: D[j] = (C[j] - P[j]) mod 26
    residues = defaultdict(list)
    for j, (pc, cc) in enumerate(zip(P, C)):
        pval = A2I[pc]
        cval = A2I[cc]
        d = (cval - pval) % 26
        residues[j % m].append(d)

    # For each residue t we may have multiple observations; they must be consistent:
    # d ≡ S_total + m*k_t (mod 26)  => so differences between observations for same residue must be ≡ 0 mod 26
    # We'll take the most common d per residue (or require consistency)
    residue_repr = {}
    for t in range(m):
        if len(residues[t]) == 0:
            residue_repr[t] = None  # no info for this residue
        else:
            # choose the value that appears most (mode) as representative
            counts = {}
            for val in residues[t]:
                counts[val] = counts.get(val, 0) + 1
            # pick the most frequent observed d
            best_d = max(counts.items(), key=lambda x: x[1])[0]
            residue_repr[t] = best_d

    # Now we have equations for residues with info:
    # residue_repr[t] ≡ S_total + m*k_t  (mod 26)  => m*k_t ≡ residue_repr[t] - S_total
    # We enumerate possible S_total in 0..25 and attempt to solve for all k_t that have info.
    candidate_keys = []

    for S_total in range(26):
        possible = True
        per_col_solutions = {}
        for t in range(m):
            d = residue_repr.get(t)
            if d is None:
                per_col_solutions[t] = list(range(26))  # unconstrained: any 0..25
            else:
                rhs = (d - S_total) % 26
                sols = solve_congruence_for_k_t(rhs, m)
                if not sols:
                    possible = False
                    break
                per_col_solutions[t] = sols
        if not possible:
            continue
        # Now combine choices across residues - cartesian product could be big if many unconstrained columns.
        # To keep it practical, if number of combinations is huge we bail or prune.
        total_combos = 1
        for t in range(m):
            total_combos *= len(per_col_solutions[t])
            if total_combos > 200000:  # safety cap
                break
        if total_combos > 200000:
            # skip this S_total (too many combos) - alternative: sample or beam-search
            continue

        # enumerate all combos
        from itertools import product
        for combo in product(*(per_col_solutions[t] for t in range(m))):
            key = list(combo)
            # verify S_total is consistent with sum(key)
            if sum(key) % 26 != S_total % 26:
                # The definition S_total = sum(k_t) (mod 26) must hold
                continue
            candidate_keys.append(key)

    # remove duplicates
    unique = []
    seen = set()
    for k in candidate_keys:
        tup = tuple(k)
        if tup not in seen:
            seen.add(tup)
            unique.append(k)
    return unique

In [ ]:
# ---------- Demo usage ----------
if __name__ == "__main__":
    # Example: create small ciphertext from earlier composite_encrypt if you want to test.
    # But here we just demonstrate recover_keys_from_known_plaintext usage.
    # Suppose m=2 and we know a segment:
    known_plain = "ONEFORALL"   # known plaintext segment (letters only)
    # You must supply the corresponding ciphertext segment (aligned).
    # If you have the entire ciphertext, extract the portion that corresponds to known_plain.
    # For demo we'll construct ciphertext using the composite encryptor (if you have it).
    # Example small composite encryptor (A->0, B->1 key 'AB'):
    def vigenere_encrypt(text, key):
        out = []
        for i,ch in enumerate(text):
            out.append(I2A[(A2I[ch] + key[i % len(key)]) % 26])
        return "".join(out)
    def composite_encrypt(plaintext, key_letters):
        text = normalize(plaintext)
        for kr in key_letters:
            text = caesar_shift_text(text, kr)
            text = vigenere_encrypt(text, key_letters)
        return text

    key_str = "AB"
    key_letters = [A2I[c] for c in key_str]  # [0,1]
    full_cipher = composite_encrypt(known_plain, key_letters)
    print("Demo ciphertext (from encryption):", full_cipher)

    # Now recover keys with known plaintext-ciphertext pair. Suppose m=2:
    candidates = recover_keys_from_known_plaintext(known_plain, full_cipher, m=2)
    print("Candidate keys (0-based ints):", candidates)
    for k in candidates:
        print("Key as letters:", ''.join(I2A[x] for x in k))
        # Verify by decrypting full_cipher:
        dec = composite_decrypt(full_cipher, k)
        print("Decrypted:", dec)


Demo ciphertext (from encryption): PQFIPUBOM
Candidate keys (0-based ints): [[0, 1], [13, 14]]
Key as letters: AB
Decrypted: ONEFORALL
Key as letters: NO
Decrypted: ONEFORALL
